In [127]:
import tensorflow as tf
import numpy as np
import keras
import pickle
from keras import layers
from tensorflow import data
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [ ]:
labellist=['label']
covlist=['subject1','Female','Age','k1a','k2a','k3a','Mono','Gran','CD4T','NK','CD8_naive','CD8pCD28nCD45RAn','PlasmaBlast']
snplist=['subject1','GSA-rs2294942_G','rs1939015_G','rs3762352_C','rs525157_A','rs72660967_T','rs6142884_A']
cpglist=['subject1','cg25929399','cg05704942','cg10306192','cg24088508','cg20089799','cg06961873','cg00704664','cg06961873']
featurelist = covlist+snplist+cpglist
fulllist = featurelist+labellist
# print('covlist',covlist)
csv_batch_size = 100
training_batch_size = 10
with open("rawdata/merge/tfrecord/feature_dict.pkl", "rb") as f:
    peg_header_dict = pickle.load(f)
peg_header_dict_subset = {}
for f in fulllist:
    if f in peg_header_dict:
        peg_header_dict_subset[f] = peg_header_dict[f]
        
print('dictionary for tfrecord:',peg_header_dict_subset)


In [376]:

# Sample string labels
labels = ['cases','controls']

# 1. Integer Encoding
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)

# 2. One-Hot Encoding
onehot_encoded = to_categorical(integer_encoded)

#print('onehot',onehot_encoded)

feature_description = peg_header_dict_subset
def parse_single(example_proto,sublist):
    # Parse the input `tf.train.Example` proto using the dictionary above.
    diction = tf.io.parse_single_example(example_proto, feature_description)
    # convert label to a float
    label = diction['label']
    intlabel = tf.constant(0.0,dtype='float32')
    if label=='cases':
        intlabel = tf.constant(1.0,dtype='float32')
    diction['label'] = intlabel
    filteredlist = {key:diction[key] for key in sublist}
    return filteredlist

def floatcaster(features):
    casted = [tf.cast(features[column], tf.float32)  for column in features if tf.debugging.is_numeric_tensor(features[column])]
    #print('casted',casted)
    features = tf.concat(  tf.transpose(casted), 1)    
    return features

def parse_batched(features,sublist):
    # Parse the input `tf.train.Example` proto using the dictionary above.
    # print('features',features)
    filteredlist = {key:features[key] for key in sublist}    
    
    #del(diction['label'])
    return filteredlist
    
def parse_batched2(features,covlist,snplist,cpglist,labellist):
    feature_dict = {}
    feature_dict['cov_input'] = floatcaster( {key:features[key] for key in covlist} )
    feature_dict['snp_input'] = floatcaster( {key:features[key] for key in snplist} )
    feature_dict['cpg_input'] = floatcaster( {key:features[key] for key in cpglist} )
    labels = features[labellist[0]]
    return (feature_dict,labels)
    



In [390]:
#filenames = ['rawdata/merge/dataset.tfrecords']
filenames = ['rawdata/merge/tfrecord/dataset.tfrecords']
raw_dataset = tf.data.TFRecordDataset(filenames)
full_dataset = raw_dataset.map(lambda x: parse_single(x,featurelist+labellist))
#print('full',full_dataset)
batched_dataset = full_dataset.repeat().shuffle(100).batch(100)
#print('batched',next(iter(batched_dataset)))
batched_dataset2 = batched_dataset.map(lambda features:parse_batched2(features,covlist,snplist,cpglist,labellist))
#print('batched2',next(iter(batched_dataset2)))
train = batched_dataset2.take(3)
val = batched_dataset2.skip(3)

print('cov',train.element_spec[0]['cov_input'].shape[1])
print('snp',train.element_spec[0]['snp_input'].shape[1])
print('cpg',train.element_spec[0]['cpg_input'].shape[1])
print('train',next(iter(train)))

cov 12
snp 6
cpg 7
train ({'cov_input': <tf.Tensor: shape=(100, 12), dtype=float32, numpy=
array([[0.00000e+00, 6.81000e+01, 4.30000e-02, ..., 1.40274e+02,
        9.59874e+00, 1.73824e+00],
       [0.00000e+00, 8.04000e+01, 4.00000e-03, ..., 1.71230e+02,
        8.14590e+00, 1.98938e+00],
       [0.00000e+00, 7.60000e+01, 1.40000e-02, ..., 1.32754e+02,
        1.43383e+01, 1.69922e+00],
       ...,
       [0.00000e+00, 7.67000e+01, 3.30000e-02, ..., 8.73735e+01,
        1.74463e+01, 1.90920e+00],
       [0.00000e+00, 7.23000e+01, 4.00000e-03, ..., 1.50850e+02,
        1.72959e+01, 1.79724e+00],
       [0.00000e+00, 6.59000e+01, 6.00000e-03, ..., 2.20976e+02,
        8.33296e+00, 1.90008e+00]], dtype=float32)>, 'snp_input': <tf.Tensor: shape=(100, 6), dtype=float32, numpy=
array([[0., 0., 2., 1., 0., 0.],
       [0., 0., 1., 2., 0., 1.],
       [1., 1., 2., 2., 1., 1.],
       [0., 1., 2., 2., 0., 0.],
       [0., 1., 1., 2., 1., 1.],
       [0., 0., 2., 1., 2., 0.],
       [1., 1., 2.

In [ ]:
print('covlist',covlist)
cov_batched = batched_dataset.map(lambda features: parse_batched(features,covlist))
print('covbatched',next(iter(cov_batched)))
snp_batched = batched_dataset.map(lambda features: parse_batched(features,snplist))
# print('snpbatched',next(iter(snp_batched)))
cpg_batched = batched_dataset.map(lambda features: parse_batched(features,cpglist))
# print('cpgbatched',next(iter(cpg_batched)))
label_batched = batched_dataset.map(lambda features: parse_batched(features,labellist))
print('labelbatched',next(iter(label_batched)))

In [ ]:
    
cov_processed = cov_batched.map(floatcaster)
snp_processed = snp_batched.map(floatcaster)
cpg_processed = cpg_batched.map(floatcaster)
label_processed = label_batched.map(floatcaster)
print('cov_processed',cov_processed)
print('sample cov',next(iter(cov_processed)))
train_batches = 4
cov_train = cov_processed.take(train_batches)
snp_train = snp_processed.take(train_batches)
cpg_train = cpg_processed.take(train_batches)
label_train = label_processed.take(train_batches)
cov_val = cov_processed.skip(train_batches)
snp_val = snp_processed.skip(train_batches)
cpg_val = cpg_processed.skip(train_batches)
label_val = label_processed.skip(train_batches)

In [361]:
print(cpg_train.element_spec.shape[1])
#for element in val:
    #print('counter val',counter,element)
    #counter+=1
# for raw_record in raw_dataset.take(1):
#   example = tf.train.Example()
#   example.ParseFromString(raw_record.numpy())
#   #print(example)

# # Create a description of the features.
# feature_description = {
#     'geno': tf.io.FixedLenFeature([], tf.int64, default_value=0),    
#     'sample': tf.io.FixedLenFeature([], tf.string, default_value=''),
#     'cpg': tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
# }

7


In [236]:
def floatcaster2(features, labels):
    casted = [tf.cast(features[column], tf.float32)  for column in snplist+cpglist]
    #print('casted',casted)
    features = tf.concat(  tf.transpose(casted), 1)

    intlabels = tf.map_fn(lambda x: 1.0 if x=='cases' else 0.0 , labels,fn_output_signature= tf.float32)
    #intlabel = 0.0
    #if labels == 'cases':
        #intlabel = 1.0        
    return features,intlabels
    
    #return (tf.stack(list(features.values()), axis=1), labels)

#dataset = dataset.map(workaround)
merged_ds = tf.data.experimental.make_csv_dataset(file_pattern="rawdata/merge/*/pasted.csv.new",num_parallel_reads=2,select_columns=labellist+snplist+cpglist,label_name='label',shuffle_buffer_size=500,batch_size=csv_batch_size,num_epochs=None).map(floatcaster2)
print(merged_ds)

<_MapDataset element_spec=(TensorSpec(shape=(100, 3), dtype=tf.float32, name=None), TensorSpec(shape=(100,), dtype=tf.float32, name=None))>


In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])

In [393]:
def create_cov_model(dataset):
    input_dim = dataset.element_spec[0]['cov_input'].shape[1]
    print('inputsize',input_dim)    
    inputs = layers.Input(shape=(input_dim,),name='cov_input')
    x = layers.Dense(1,name='cov_output')(inputs)
    return keras.models.Model(inputs,x,name='cov_model')
cov_model =create_cov_model(train)
print(cov_model.summary())

def create_snp_model(dataset):
    input_dim = dataset.element_spec[0]['snp_input'].shape[1]
    print('inputsize',input_dim)    
    inputs = layers.Input(shape=(input_dim,),name='snp_input')
    x = layers.Dense(32,name='snp_gene')(inputs)
    x = layers.Dense(1,name='snp_gene_output')(x)
    return keras.models.Model(inputs,x,name='snp_model')
snp_model = create_snp_model(train)
print(snp_model.summary())

def create_cpg_model(dataset):
    input_dim = dataset.element_spec[0]['cpg_input'].shape[1]
    print('inputsize',input_dim)    
    inputs = layers.Input(shape=(input_dim,),name='cpg_input')
    x = layers.Dense(32,name='cpg_gene')(inputs)
    x = layers.Dense(1,name='cpg_gene_output')(x)
    return keras.models.Model(inputs,x,name='cpg_model')
cpg_model = create_cpg_model(train)
print(cpg_model.summary())

def create_pathway_model(cov_model,snp_model,cpg_model):
    pathwayInput = layers.concatenate([snp_model.output,cpg_model.output],name='pathway_input')
    x = layers.Dense(8,name='pathway_level_1')(pathwayInput)
    x = layers.Dense(4,name='pathway_level_2')(x)
    x = layers.concatenate([cov_model.output,x],name='pathway_output')
    x = layers.Dense(1)(x)
    #return keras.models.Model(inputs=[cov_model.input,snp_model.input,cpg_model.input],outputs=x)
    return keras.models.Model(inputs={'cov_input':cov_model.input,'snp_input':snp_model.input,'cpg_input':cpg_model.input},outputs=x)
pathway_model = create_pathway_model(cov_model,snp_model,cpg_model)
print(pathway_model.summary())
print(pathway_model.inputs)

inputsize 12


Model: "cov_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ cov_input (InputLayer)          │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cov_output (Dense)              │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13 (52.00 B)

 Trainable params: 13 (52.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ snp_input           │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cpg_input           │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ snp_gene (Dense)    │ (None, 32)        │        224 │ snp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cpg_gene (Dense)    │ (None, 32)        │        256 │ cpg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ snp_gene_output     │ (None, 1)         │         33 │ snp_gene[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cpg_gene_output     │ (None, 1)         │         33 │ cpg_gene[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pathway_input       │ (None, 2)         │          0 │ snp_gene_output[… │
│ (Concatenate)       │                   │            │ cpg_gene_output[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cov_input           │ (None, 12)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pathway_level_1     │ (None, 8)         │         24 │ pathway_input[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cov_output (Dense)  │ (None, 1)         │         13 │ cov_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pathway_level_2     │ (None, 4)         │         36 │ pathway_level_1[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pathway_output      │ (None, 5)         │          0 │ cov_output[0][0], │
│ (Concatenate)       │                   │            │ pathway_level_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_55 (Dense)    │ (None, 1)         │          6 │ pathway_output[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 625 (2.44 KB)

 Trainable params: 625 (2.44 KB)

 Non-trainable params: 0 (0.00 B)

None
[<KerasTensor shape=(None, 12), dtype=float32, sparse=False, name=cov_input>, <KerasTensor shape=(None, 7), dtype=float32, sparse=False, name=cpg_input>, <KerasTensor shape=(None, 6), dtype=float32, sparse=False, name=snp_input>]


In [397]:
#print(next(iter(cov_train)).numpy())
pathway_model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
pathway_model.fit(train ,validation_data=val,steps_per_epoch=3,validation_steps=2,epochs=5)
#model.fit(train,validation_data=val,steps_per_epoch=4,validation_steps=3,epochs=5)

Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 27s 10s/step - accuracy: 0.8421 - loss: 2.5453 - val_accuracy: 0.1150 - val_loss: 14.2645
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7650 - val_loss: 3.7878
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.8246 - loss: 2.8274 - val_accuracy: 0.3200 - val_loss: 10.9603
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2100 - val_loss: 12.7333
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.8350 - loss: 2.6595 - val_accuracy: 0.7750 - val_loss: 3.6266


In [142]:
FEATURE_NAMES=snplist+cpglist
categorylist=[]
def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = tf.keras.layers.Normalization(name=name)
    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))
    # Learn the statistics of the data
    normalizer.adapt(feature_ds)
    # Normalize the input feature
    encoded_feature = normalizer(feature)
    print('encoded_feature',encoded_feature)
    return encoded_feature
def create_model_inputs():
    inputs = {}

    # This a helper function for creating categorical features
    def create_input_helper(feature_name):
        num_categories = 3
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(num_categories,), dtype="int64"
        )
        return inputs

    for feature_name in FEATURE_NAMES:
        if feature_name in categorylist:
            # Categorical features
            create_input_helper(feature_name)
        else:
            # Make them float32, they are Real numbers
            feature_input = tf.keras.layers.Input(name=feature_name, shape=(1,))
            # Process the Inputs here
            print('feature_input',feature_input)
            #inputs[feature_name] = encode_numerical_feature(
             #    feature_input, feature_name, batched
            #inputs[feature_name] = feature_input
            
    return inputs
def create_model_inputs2():
    inputs = {}
    for snp in snplist:
        inputs[snp] = tf.keras.layers.Input(name=snp, shape=(1,),dtype='int64')
    for cpg in cpglist:
        inputs[cpg] = tf.keras.layers.Input(name=snp, shape=(1,),dtype='float32')
    return inputs
inputlist = create_model_inputs2()
print(inputlist)

{'rs138985554_T': <KerasTensor shape=(None, 1), dtype=int64, sparse=False, name=rs138985554_T>, 'rs4648601_C': <KerasTensor shape=(None, 1), dtype=int64, sparse=False, name=rs4648601_C>, 'cg00050873': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=rs4648601_C>}


In [226]:
#inputs = tf.keras.Input(shape=(3,),name=snplist+cpglist)

class MyModel(keras.Model):
    def __init__(self):
        super().__init__()
        
        self.dense1 = keras.layers.Dense(32, activation="relu",name='first')
        self.dense2 = keras.layers.Dense(1,name='out')
        self.dropout = keras.layers.Dropout(0.5,name='dropout')

    def call(self, inputs, training=False):
        #all_features = layers.concatenate(list(inputs.values()))
        x = self.dense1(inputs)
        #x = self.dense1(all_features)
        x = self.dropout(x, training=training)
        return self.dense2(x)

class Classifier(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.dense_1 = layers.Dense(32, activation="relu")
        self.dropout = layers.Dropout(0.5)
        self.dense_2 = layers.Dense(1)

    def call(self, inputs):
        #all_features = layers.concatenate(list(inputs.values()))
        x = self.dense_1(inputs)
        #x = self.dense_1(all_features)
        x = self.dropout(x)
        output = self.dense_2(x)
        return output

    # Surpress build warnings
    def build(self, input_shape):
        self.built = True

def create_model():
    all_inputs = create_model_inputs2()
    print(inputlist)
    output = Classifier()(all_inputs)
    model = keras.Model(all_inputs,output)
    #model = MyModel()
    return model

input_dim = merged_ds.element_spec[0].shape[1]

# version 1
#model = create_model()
#all_inputs = 

#version 2
#model = MyModel()

x = keras.layers.Input(shape=(input_dim,))
output = model(x)

model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
print(model.summary())
print('shape',output.shape)
print('allinputs',all_inputs)
#print(output.summary())


Model: "my_model_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ first (Dense)                   │ (None, 32)             │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ out (Dense)                     │ (None, 1)              │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 161 (644.00 B)

 Trainable params: 161 (644.00 B)

 Non-trainable params: 0 (0.00 B)

None
shape (None, 1)
allinputs {'rs138985554_T': <KerasTensor shape=(None, 1), dtype=int64, sparse=False, name=rs138985554_T>, 'rs4648601_C': <KerasTensor shape=(None, 1), dtype=int64, sparse=False, name=rs4648601_C>, 'cg00050873': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=rs4648601_C>}


In [ ]:
#train,val = tf.keras.utils.split_dataset(merged_ds,left_size=0.7)
train,val = tf.keras.utils.split_dataset(batched2,left_size=0.7)

In [265]:
#train
#for i in train:
#model.fit(i, epochs=50, validation_data=val)
#model.fit(batched, epochs=50)
#print(merged_ds.dims[1])

#unbatched = merged_ds.unbatch()
#rebatched = unbatched.batch(20)
#print(unbatched)
#print(unbatched.batch(20))
#expanded_input = tf.expand_dims(merged_ds.batch(20), axis=-1)
#print(expanded_input)
#model.fit(merged_ds)
model.fit(train,validation_data=val,steps_per_epoch=4,validation_steps=3,epochs=5)
#print('train',train,'val',val)



Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 31s 9s/step - accuracy: 0.4893 - loss: 3.9107 - val_accuracy: 0.4667 - val_loss: 0.7243
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5633 - val_loss: 0.7103
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.5467 - loss: 2.6293 - val_accuracy: 0.4867 - val_loss: 0.7005
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5367 - val_loss: 0.7446
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 4s/step - accuracy: 0.5000 - loss: 2.8988 - val_accuracy: 0.5067 - val_loss: 0.6945


In [260]:
model.summary()

Model: "my_model_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ first (Dense)                   │ (None, 32)             │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ out (Dense)                     │ (None, 1)              │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 485 (1.90 KB)

 Trainable params: 161 (644.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 324 (1.27 KB)